# @robotodio - tweet extractor

---

## Libraries

In [15]:
# Python libraries
# ------------------------------------------------------------------------------
# Reading files with different formats
import json

# Data wrangling
import re
import pandas as pd
import numpy as np

# Twitter API
import tweepy

# NLP
from textblob import TextBlob
from googletrans import Translator

# Hate speech detection
from detoxify import Detoxify

## 1. Download data from twitter

The tweepy API class provides access to the entire Twitter RESTful API methods. Each method can accept various parameters and return responses to the different requests.

### 1.1. API set up

In order for you to get the Twitter feed working you need four keys: 

- Consumer Key
- Consumer Secret
- Access Token
- Access Token Secret

If you have a developer Twitter account you can generate your own tokens and log in with them. In case you don't have a developer account, please contact me and I will provide mine.

In [2]:
# API Twitter credentials
# ------------------------------------------------------------------------------
# Open .json file containing credentials/tokens as a dictionary
with open("../twitter_api_keys.json") as file:
    api_credentials = json.load(file)
    
# Assign each value of the dictionary to a new variable
consumer_key = api_credentials['consumer_key']
consumer_secret = api_credentials['consumer_secret']
access_token = api_credentials['access_token']
access_token_secret = api_credentials['access_token_secret']

In [3]:
# API set up
# ------------------------------------------------------------------------------
# Create a handler instance with key and secret consumer, and pass the tokens
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
    
# Construct the API instance
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Check credentials
if(api.verify_credentials):
    print("Logged In Successfully.")
else:
    print("Error -- Could not log in with your credentials.")

Logged In Successfully.


### 1.2. Extract tweets and associated metadata

Once the `tweepy.API` class has been instantiated we can call its methods to scrap desired information in Twitter.

Pagination is used a lot in Twitter API development. Iterating through timelines, user lists, direct messages, etc. In order to perform pagination, we must supply a page/cursor parameter with each of our requests. The problem here is this requires a lot of boiler plate code just to manage the pagination loop. To help make pagination easier and require less code, Tweepy has the Cursor object.

In first place, we are going to define a function that returns an iterator of tweets. We will call this fuction each time we want to download tweets from an user account.

In [4]:
# Tweets list iterator
# ------------------------------------------------------------------------------
def tweets_iterator(target, n_items):
    '''
    Returns an iterator of tweets.

        Parameters:
            target (str): The user name of the Twitter account.
            n_items (int): Number of tweets downloaded.

        Returns:
            tweets (ItemIterator): an iterator of tweets.
    '''
    # Instantiate the iterator
    tweets = tweepy.Cursor(
        api.user_timeline,
        screen_name=target,
        include_rts=False,
        exclude_replies=False,
        tweet_mode='extended').items(n_items)
    
    # Returns iterator
    return tweets

In the next cell we are going to download $n$ tweets from a Twitter account, and examinate the content of the first tweet with all the metadata associated.

In [5]:
# Introduce the target Twitter account and number of items to download
target = 'DFD_74'
n_items = 150

# Tweets list (iterator)
tweets = tweets_iterator(target, n_items=1)

# Print the first tweet with .json format
print(json.dumps(next(tweets)._json, indent=4))

{
    "created_at": "Tue Mar 02 17:24:37 +0000 2021",
    "id": 1366801630918828032,
    "id_str": "1366801630918828032",
    "full_text": "@VSocialGob M\u00e1s chiringuitos de mierda para sacarnos el dinero\nDais asco",
    "truncated": false,
    "display_text_range": [
        12,
        72
    ],
    "entities": {
        "hashtags": [],
        "symbols": [],
        "user_mentions": [
            {
                "screen_name": "VSocialGob",
                "name": "Vicepresidencia de Derechos Sociales y Agenda 2030",
                "id": 1215200964375994373,
                "id_str": "1215200964375994373",
                "indices": [
                    0,
                    11
                ]
            }
        ],
        "urls": []
    },
    "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>",
    "in_reply_to_status_id": 1366739487934578690,
    "in_reply_to_status_id_str": "1366739487934578690",
    "in_reply_to_use

As we can see in the previous output cell, each tweet contains a lot of metadata. All this information comes in .json format, and we can access to it with Tweepy methods, or with dictionary methods.

In the next cell we are going to extract some metadata from each tweet, and export to a Pandas DataFrame format.

In [7]:
# Tweet extractor
# ------------------------------------------------------------------------------
# Tweets list (iterator)
tweets = tweets_iterator(target, n_items)

# Read through the iterator, and export the info to a Pandas DataFrame
all_columns = [np.array([
    tweet.full_text,
    tweet.user.screen_name,
    tweet.id,
    tweet.user.followers_count,
    tweet.source,
    tweet.created_at,
    len(tweet.full_text),
    tweet.favorite_count,
    tweet.retweet_count,
    str(tweet.entities['hashtags'])
]) for tweet in tweets]

# Export the list of tweets to a dataframe
df = pd.DataFrame(
    data=all_columns,
    columns=['tweet', 'account', 'id', 'followers', 'source', 'date', 'length',
              'likes', 'RTs', 'hashtags'])

df.head()

,tweet,account,id,followers,source,date,length,likes,RTs,hashtags
0,@VSocialGob Más chiringuitos de mierda para sa...,DFD_74,1366801630918828032,5788,Twitter for Android,2021-03-02 17:24:37,72,0,0,[]
1,-6 millones de parados \n\nQue viene la ultra...,DFD_74,1366791489033023495,5788,Twitter for Android,2021-03-02 16:44:19,226,12,6,[]
2,"6 millones de parados y sonríes? \n\nClaro, no...",DFD_74,1366789932279332873,5788,Twitter for Android,2021-03-02 16:38:08,78,12,5,[]
3,Cada vez que gobierna el PSOE bajamos 2 escalo...,DFD_74,1366787944271532033,5788,Twitter for Android,2021-03-02 16:30:14,154,4,0,[]
4,PSOE = Paro \n\nSe entiende verdad?,DFD_74,1366786558842576907,5788,Twitter for Android,2021-03-02 16:24:43,33,30,6,[]


In addition, we are going to create another dataframe with the main data of the target account:

In [8]:
# Account data extractor
# ------------------------------------------------------------------------------
# Tweets list (iterator)
tweets = tweets_iterator(target, n_items=1)

# Read through the iterator, and export the info to a Pandas DataFrame
data_account = [np.array([
    tweet.user.screen_name,
    tweet.user.name,
    tweet.user.description,
    tweet.user.created_at,
    tweet.user.friends_count,
    tweet.user.followers_count,
    tweet.user.statuses_count
]) for tweet in tweets]

# Export the list of features to a dataframe
df_account = pd.DataFrame(
    data=data_account,
    columns=['account', 'account_name', 'bio_description', 'creation_date', 'friends',
             'followers', 'tweets']
)

df_account = pd.melt(df_account)
df_account

,variable,value
0,account,DFD_74
1,account_name,Español74 (david) 🇪🇸
2,bio_description,"Padre orgulloso, marido enamorado, currante, e..."
3,creation_date,2017-09-15 09:52:18
4,friends,588
5,followers,5788
6,tweets,61288


### 1.3. Clean and translation

Once we have a dataset with all the tweets of an user, we need to prepare and clean them to serve as input to the model. For each tweet we are going to eliminate special string characters, and we will translate them into English in case they are not.

The traduction is needed for two mains reasons:

- Make a more robust and extensible app
- Detoxify model provides more information when the input is in english language

**Cleaning:**

In [9]:
# Data cleaning
# ------------------------------------------------------------------------------
# Characters to remove
spec_chars = ['\n', '\t', '\r']

# Replace defined characters with a whitespace
for char in spec_chars:
    df['tweet'] = df['tweet'].str.strip().replace(char, ' ')

Because we replaced the special characters with a whitespace, we might end up with double whitespaces in some values. Let’s remove them by splitting each tweet using whitespaces and re-joining the words again using join.

In [10]:
# Split and re-join each tweet
df['tweet'] = df['tweet'].str.split().str.join(" ")

In [11]:
df.head()

,tweet,account,id,followers,source,date,length,likes,RTs,hashtags
0,@VSocialGob Más chiringuitos de mierda para sa...,DFD_74,1366801630918828032,5788,Twitter for Android,2021-03-02 17:24:37,72,0,0,[]
1,-6 millones de parados Que viene la ultraderec...,DFD_74,1366791489033023495,5788,Twitter for Android,2021-03-02 16:44:19,226,12,6,[]
2,"6 millones de parados y sonríes? Claro, no gob...",DFD_74,1366789932279332873,5788,Twitter for Android,2021-03-02 16:38:08,78,12,5,[]
3,Cada vez que gobierna el PSOE bajamos 2 escalo...,DFD_74,1366787944271532033,5788,Twitter for Android,2021-03-02 16:30:14,154,4,0,[]
4,PSOE = Paro Se entiende verdad?,DFD_74,1366786558842576907,5788,Twitter for Android,2021-03-02 16:24:43,33,30,6,[]


**Translation:**

Finally we translate them into English. For carrying out this task, we are going to use the TextBlob library, which is a module for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more.

In the next cell we can see an example of translation:

In [20]:
translator = Translator()
translator.translate('안녕하세요.')

AttributeError: 'NoneType' object has no attribute 'group'

In [18]:
# Translation example
text = "Esta frase está en español y queremos traducirla al inglés."
translated_text = TextBlob(text).translate(to="en").string
translated_text

HTTPError: HTTP Error 429: Too Many Requests

In [ ]:
'''
# Main language detection
# ------------------------------------------------------------------------------
# Detect the language pf each tweet
df['org_language'] = df['tweet'].apply(
    lambda tweet: TextBlob(tweet).detect_language())

# Summary of the different languages
freq_languages = df[['id', 'org_language']].groupby(["org_language"])\
                                           .count()\
                                           .sort_values(["org_language"])\
                                           .reset_index()

# Most common language
main_language = freq_languages.values[0][0]
print(f"The most common language in @{target} account is {main_language}.")
'''

Due to limitations with the TextBlob library and to avoiding be banned, we are going to introduce manually the tweets' origin language.

In [27]:
# Translation
# ------------------------------------------------------------------------------
# Instance a np.array variable that contains all tweets
tweets = np.array(df['tweet'])

# We apply the TextBlob translation to each tweet of the previous np.array
tweets_en = list(map(lambda tweet: TextBlob(tweet).translate(from_lang='es', to='en').string, tweets))

# Create another pandas column with the transalation values
df['tweet_en'] = tweets_en

In [28]:
df.head()

,tweet,account,id,followers,source,date,length,likes,RTs,hashtags,org_language,tweet_en
0,@sanchezdelreal @davidgilphoto @_InakiLopez_ V...,DFD_74,1365687356028780546,5792,Twitter for Android,2021-02-27 15:36:53,127,2,0,[],es,@sanchezdelreal @davidgilphoto @_InakiLopez_ G...
1,Nunca escuché a Irene Montero salir a defender...,DFD_74,1365685600947490818,5792,Twitter for Android,2021-02-27 15:29:55,203,42,14,[],es,I never heard Irene Montero go out to defend A...
2,@alejandroTGN No señor. Le doy la vuelta. La n...,DFD_74,1365663812922331137,5792,Twitter for Android,2021-02-27 14:03:20,120,34,6,[],es,@alejandroTGN No sir. I turn it over. The disa...
3,@isaiiki Echarle cojones,DFD_74,1365663117280215050,5792,Twitter for Android,2021-02-27 14:00:34,24,5,0,[],es,@isaiiki Fuck him
4,@LaMuy Eres despreciable A asco das Miserable,DFD_74,1365574559055962113,5792,Twitter for Android,2021-02-27 08:08:40,47,2,0,[],es,@LaMuy You are despicable A disgusting you are...


## 2. Hate speech level prediction

`Detoxify` library has three pre-trained models:

- original: toxic, severe_toxic, obscene, threat, insult, identity_hate.
- unbiased: toxicity, severe_toxicity, obscene, threat, insult, identity_attack, sexual_explicit.
- multilingual: toxicity

While *original* and *unbiased* models generate different scorings like: toxicity, obscene, threat, insult...etc, *multilingual* model only produces a toxicity scoring. On the other hand, *multilingual* model is the only one that can bring a toxicity score in a language different from english.

Thanks to translation, we can use *original* or *unbiased* model independently from the tweets origin language.

In [29]:
# Hate speech level prediction
# ------------------------------------------------------------------------------
# Returns a dictionary with toxicity values of each tweet. The keys of the
# dictionary are called 'toxicity', 'obscene'...etc
results = Detoxify('original').predict(list(df['tweet_en']))
results

{'toxicity': [0.24685078859329224,
  0.5825507640838623,
  0.0007241822895593941,
  0.9965693950653076,
  0.9647840261459351,
  0.973308801651001,
  0.0022345962934195995,
  0.7605370879173279,
  0.013281521387398243,
  0.01974884420633316,
  0.9745989441871643,
  0.6810771822929382,
  0.009596984833478928,
  0.9319608211517334,
  0.0009022983140312135,
  0.0014949142932891846,
  0.8648038506507874,
  0.0031050562392920256,
  0.22857268154621124,
  0.0007731550722382963],
 'severe_toxicity': [0.00022423798509407789,
  0.0005660558817908168,
  0.00010977540659951046,
  0.2972474694252014,
  0.006988096982240677,
  0.08976218849420547,
  9.09119044081308e-05,
  0.0018584170611575246,
  9.865137690212578e-05,
  0.00010276897228322923,
  0.04157593846321106,
  0.003911942709237337,
  0.00010719172860262915,
  0.022784337401390076,
  0.00010690357157727703,
  9.219614003086463e-05,
  0.007149782497435808,
  8.701199112692848e-05,
  0.0007667197496630251,
  0.00010751767695182934],
 'obscene

In [30]:
# Add the new info to the previous DataFrame
df['toxicity'] = results['toxicity']
df['obscene'] = results['obscene']
df['threat'] = results['threat']
df['insult'] = results['insult']
df['identity_hate'] = results['identity_hate']

df.head()

,tweet,account,id,followers,source,date,length,likes,RTs,hashtags,org_language,tweet_en,toxicity,obscene,threat,insult,identity_hate
0,@sanchezdelreal @davidgilphoto @_InakiLopez_ V...,DFD_74,1365687356028780546,5792,Twitter for Android,2021-02-27 15:36:53,127,2,0,[],es,@sanchezdelreal @davidgilphoto @_InakiLopez_ G...,0.246851,0.003210,0.000634,0.013238,0.000815
1,Nunca escuché a Irene Montero salir a defender...,DFD_74,1365685600947490818,5792,Twitter for Android,2021-02-27 15:29:55,203,42,14,[],es,I never heard Irene Montero go out to defend A...,0.582551,0.005755,0.004367,0.072209,0.002257
2,@alejandroTGN No señor. Le doy la vuelta. La n...,DFD_74,1365663812922331137,5792,Twitter for Android,2021-02-27 14:03:20,120,34,6,[],es,@alejandroTGN No sir. I turn it over. The disa...,0.000724,0.000180,0.000113,0.000175,0.000134
3,@isaiiki Echarle cojones,DFD_74,1365663117280215050,5792,Twitter for Android,2021-02-27 14:00:34,24,5,0,[],es,@isaiiki Fuck him,0.996569,0.989913,0.004178,0.666185,0.006640
4,@LaMuy Eres despreciable A asco das Miserable,DFD_74,1365574559055962113,5792,Twitter for Android,2021-02-27 08:08:40,47,2,0,[],es,@LaMuy You are despicable A disgusting you are...,0.964784,0.165025,0.001558,0.830454,0.008878


Once we have a score for each tweet, we can aggregate these puntuactions by calculating the average.

In [44]:
sum(df['toxicity'])/len(df)

0.41287379481073005

In [43]:
df['toxicity'].mean()

0.41287379481073005

In [41]:
scoring_average = {'variable': ['avg_toxicity',
                                'avg_obscene', 
                                'avg_threat',
                                'avg_insult',
                                'avg_identity_hate'],
                   
                   'value': [sum(df['toxicity'])/len(df),
                             sum(df['obscene'])/len(df),
                             sum(df['threat'])/len(df),
                             sum(df['insult'])/len(df),
                             sum(df['identity_hate'])/len(df)]}

df_average = pd.DataFrame(scoring_average)
df_average

,variable,value
0,avg_toxicity,0.412874
1,avg_obscene,0.183974
2,avg_threat,0.010740
3,avg_insult,0.250522
4,avg_identity_hate,0.044461


In [64]:
df = pd.concat([df_account, df_average])
df

,variable,value
0,account,DFD_74
1,name,Español74 (david) 🇪🇸
2,description,"Padre orgulloso, marido enamorado, currante, e..."
3,creation_date,2017-09-15 09:52:18
4,friends,588
5,followers,5790
6,tweets,61215
0,avg_toxicity,0.352327
1,avg_obscene,0.134397
2,avg_threat,0.010379


## 3. Data explotation

## X. Bibliografía

- http://docs.tweepy.org/en/latest/
- https://github.com/unitaryai/detoxify

## X. Info a revisar

- Limpieza de datos con regex: http://rios.tecnm.mx/cdistribuido/recursos/MinDatScr/MineriaScribble.html
- Web objetivo: https://www.ninjalitics.com/


- Traduction --> TextBlob
- Sentiment Analysis --> TextBlob
- Universal Dependencies --> GSD

Preparar un traductor con TextBlob:

TextBlob también tiene análisis de opinión:

In [ ]:
https://blog.usejournal.com/why-and-how-to-make-a-requirements-txt-f329c685181e